In [1]:
# Task 1 Assignment 2
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices: tf.config.experimental.set_memory_growth(device, True)

from tensorflow import keras
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Flatten
#from tensorflow.keras.layers import Dense
#from tensorflow.keras.layers import Dropout
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

tf.random.set_seed(4)


In [2]:
# Make a choice of dataset, mnist (0) or fashion_mnist (1)
choice = 0

if choice == 0:
    dataset = keras.datasets.mnist
    class_names = ["0","1","2","3","4","5","6","7","8","9"]
elif choice == 1:
    dataset = keras.datasets.fashion_mnist
    class_names_f = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker",
                     "Bag","Ankle Boot"]
    
(X_train_full, y_train_full), (X_test, y_test) = dataset.load_data()

In [3]:
# pre-processing step

k = 5000
mx = np.max(X_train_full)

X_valid, X_train = X_train_full[:k] / mx, X_train_full[k:] / mx
y_valid, y_train = y_train_full[:k], y_train_full[k:]

# format CNN
X_test_, X_valid_, X_train_ = np.expand_dims(X_test, axis=-1), np.expand_dims(X_valid, axis=-1),np.expand_dims(X_train, axis=-1)



In [4]:
# Initialize the settings for the MLP / CNN 

# Settings regarding model structure and optimization
settings_1 = [(0,0,0,0), # base line model: MLP / CNN
              (1,0,0,0), # optimizer: ADAM
              (0,1,0,0), # dropout: including / excluding
              (0,0,1,0), # add another layer to the model
              (0,0,0,1)] # remove layer from the model

# Settings regarding the activation functions and the initialization
settings_2 = [("tanh","glorot_uniform"),
              ("tanh","glorot_normal"),
              ("relu","he_uniform"),
              ("relu","he_normal"),
              ("selu","lecun_uniform"),
              ("selu","lecun_normal")]


In [ ]:
# MLP settings 1.

runs = 10
output_1 = np.zeros([runs, 3*len(settings_1)])

for ix, s in enumerate(settings_1):
    print("Model Setting: "+str(ix))
    
    for i in range(runs):
        
        model = keras.models.Sequential()
        model.add(keras.layers.Flatten(input_shape=X_train.shape[1:3]))
        
        if not s[1]:
            model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="glorot_uniform"))
            if s[2]:
                model.add(keras.layers.Dense(200, activation="relu", kernel_initializer="glorot_uniform"))
            if not s[3]:
                model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="glorot_uniform"))
            model.add(keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_uniform"))
        else:
            model.add(keras.layers.Dropout(rate=0.2))
            model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="glorot_uniform"))
            model.add(keras.layers.Dropout(rate=0.2))
            if s[2]:
                model.add(keras.layers.Dense(200, activation="relu", kernel_initializer="glorot_uniform"))
                model.add(keras.layers.Dropout(rate=0.2))
            if not s[3]:
                model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="glorot_uniform"))
                model.add(keras.layers.Dropout(rate=0.2))
            model.add(keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_uniform"))
            model.add(keras.layers.Dropout(rate=0.2))
        
        opt = "sgd"
        if s[0]: 
            opt = "adam"
        
        model.compile(loss="sparse_categorical_crossentropy",
                      optimizer=opt,
                      metrics=["accuracy"])
        
        print(X_train.shape)
        history = model.fit(X_train, y_train, epochs=100, validation_data=(X_valid,y_valid),
                            callbacks=[keras.callbacks.EarlyStopping(patience=6)])
        
        output_1[i,ix*3] = model.evaluate(X_test, y_test)[1]
        output_1[i,3*ix+1] = len(history.history['loss'])
        output_1[i,3*ix+2] = history.history['val_accuracy'][0]



Model Setting: 0
(55000, 28, 28)
Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 3s 54us/sample - loss: 0.6080 - accuracy: 0.8450 - val_loss: 0.3084 - val_accuracy: 0.9130
Epoch 2/100
55000/55000 [==============================] - 3s 48us/sample - loss: 0.2890 - accuracy: 0.9170 - val_loss: 0.2392 - val_accuracy: 0.9328
Epoch 3/100
55000/55000 [==============================] - 3s 48us/sample - loss: 0.2362 - accuracy: 0.9322 - val_loss: 0.2045 - val_accuracy: 0.9456
Epoch 4/100
55000/55000 [==============================] - 3s 48us/sample - loss: 0.2020 - accuracy: 0.9421 - val_loss: 0.1769 - val_accuracy: 0.9514
Epoch 5/100
55000/55000 [==============================] - 3s 48us/sample - loss: 0.1757 - accuracy: 0.9494 - val_loss: 0.1568 - val_accuracy: 0.9584
Epoch 6/100
25472/55000 [============>.................] - ETA: 1s - loss: 0.1579 - accuracy: 0.9549

In [159]:
# save the obtained output
np.save('output_1.npy', output_1)

(1, 15)

In [101]:
# MLP settings 2

runs = 2
output_2 = np.zeros([runs, 3*len(settings_2)])

for ix, s in enumerate(settings_2):
    print("Activation: "+s[0]+". Initialization: "+s[1])
    
    for i in range(runs):
        print("Model iteration: "+str(i))
        model = keras.models.Sequential([
            keras.layers.Flatten(input_shape=X_train.shape[1:3]),
            keras.layers.Dense(300, activation=s[0], kernel_initializer=s[1]),
            keras.layers.Dense(100, activation=s[0], kernel_initializer=s[1]),
            keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_uniform")
        ])
        
        model.compile(loss="sparse_categorical_crossentropy",
                      optimizer="sgd",
                      metrics=["accuracy"])
        
        print(X_train.shape)
        history = model.fit(X_train, y_train, epochs=100, validation_data=(X_valid,y_valid),
                            callbacks=[keras.callbacks.EarlyStopping(patience=6)])
        
        output_2[i,ix*3] = model.evaluate(X_test, y_test)[1]
        output_2[i,3*ix+1] = len(history.history['loss'])
        output_2[i,3*ix+2] = history.history['val_accuracy'][0]


Activation: tanh. Initialization: glorot_uniform
Model iteration: 0
(55000, 28, 28)
Train on 55000 samples, validate on 5000 samples
Epoch 1/2
55000/55000 [==============================] - 3s 51us/sample - loss: 0.5539 - accuracy: 0.8590 - val_loss: 0.3280 - val_accuracy: 0.9100
Epoch 2/2
10000/10000 [==============================] - 0s 36us/sample - loss: 0.2948 - accuracy: 0.9154
Model iteration: 1
(55000, 28, 28)
Train on 55000 samples, validate on 5000 samples
Epoch 1/2
55000/55000 [==============================] - 3s 52us/sample - loss: 0.5465 - accuracy: 0.8608 - val_loss: 0.3278 - val_accuracy: 0.9114
Epoch 2/2
10000/10000 [==============================] - 0s 35us/sample - loss: 0.2942 - accuracy: 0.9123
Activation: tanh. Initialization: glorot_normal
Model iteration: 0
(55000, 28, 28)
Train on 55000 samples, validate on 5000 samples
Epoch 1/2
55000/55000 [==============================] - 3s 50us/sample - loss: 0.5603 - accuracy: 0.8548 - val_loss: 0.3317 - val_accuracy: 0.

In [ ]:
# save the obtained output
np.save('output_2.npy', output_2)

In [149]:
# CNN settings 1

runs = 1
output_3 = np.zeros([runs, 3*len(settings_1)])

import time


for ix, s in enumerate(settings_1):
    print("Model Setting: "+str(ix))
    
    for i in range(runs):
        
        model = keras.models.Sequential()
        model.add(keras.layers.Conv2D(64, 7, activation="relu", padding="same", input_shape=[28, 28, 1]))
        model.add(keras.layers.MaxPooling2D(2))
        model.add(keras.layers.Conv2D(128, 3, activation="relu", padding="same"))
        model.add(keras.layers.Conv2D(128, 3, activation="relu", padding="same"))
        model.add(keras.layers.MaxPooling2D(2))
        if not s[3]:
            model.add(keras.layers.Conv2D(256, 3, activation="relu", padding="same"))
            model.add(keras.layers.Conv2D(256, 3, activation="relu", padding="same"))
            model.add(keras.layers.MaxPooling2D(2))
        if s[2]:
            model.add(keras.layers.Conv2D(512, 3, activation="relu", padding="same"))
            model.add(keras.layers.Conv2D(512, 3, activation="relu", padding="same"))
            model.add(keras.layers.MaxPooling2D(2))
        model.add(keras.layers.Flatten())
        
        if not s[1]:
            model.add(keras.layers.Dense(128, activation="relu"))
            model.add(keras.layers.Dropout(0.5))
            model.add(keras.layers.Dense(64, activation="relu"))
            model.add(keras.layers.Dropout(0.5))
            model.add(keras.layers.Dense(10, activation="softmax"))
            
        else:
            model.add(keras.layers.Dense(128, activation="relu"))
            model.add(keras.layers.Dense(64, activation="relu"))
            model.add(keras.layers.Dense(10, activation="softmax"))
        
        
        
        opt = "sgd"
        if s[0]: 
            opt = "adam"
        
        model.compile(loss="sparse_categorical_crossentropy",
                      optimizer=opt,
                      metrics=["accuracy"])
        
        t1 = time.time()
        history = model.fit(X_train_, y_train, epochs=100, validation_data=(X_valid_,y_valid),
                            callbacks=[keras.callbacks.EarlyStopping(patience=5)])
        print(time.time() - t1)
        
        output_3[i,ix*3] = model.evaluate(X_test_, y_test)[1]
        output_3[i,3*ix+1] = len(history.history['loss'])
        output_3[i,3*ix+2] = history.history['val_accuracy'][0]



Model Setting: 0
Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 7s 133us/sample - loss: 1.1654 - accuracy: 0.6051 - val_loss: 0.1770 - val_accuracy: 0.9508
Epoch 2/100
55000/55000 [==============================] - 7s 128us/sample - loss: 0.2801 - accuracy: 0.9208 - val_loss: 0.0804 - val_accuracy: 0.9774
Epoch 3/100
55000/55000 [==============================] - 7s 128us/sample - loss: 0.1803 - accuracy: 0.9517 - val_loss: 0.0605 - val_accuracy: 0.9830
Epoch 4/100
55000/55000 [==============================] - 7s 128us/sample - loss: 0.1385 - accuracy: 0.9636 - val_loss: 0.0580 - val_accuracy: 0.9844
Epoch 5/100
55000/55000 [==============================] - 7s 128us/sample - loss: 0.1146 - accuracy: 0.9701 - val_loss: 0.0487 - val_accuracy: 0.9862
Epoch 6/100
55000/55000 [==============================] - 7s 129us/sample - loss: 0.0983 - accuracy: 0.9748 - val_loss: 0.0534 - val_accuracy: 0.9858
Epoch 7/100
55000/55000 [===

55000/55000 [==============================] - 13s 235us/sample - loss: 1.4803 - accuracy: 0.4863 - val_loss: 0.2441 - val_accuracy: 0.9262
Epoch 2/100
55000/55000 [==============================] - 13s 228us/sample - loss: 0.3107 - accuracy: 0.9144 - val_loss: 0.1073 - val_accuracy: 0.9718
Epoch 3/100
55000/55000 [==============================] - 13s 230us/sample - loss: 0.1745 - accuracy: 0.9555 - val_loss: 0.0697 - val_accuracy: 0.9804
Epoch 4/100
55000/55000 [==============================] - 13s 229us/sample - loss: 0.1253 - accuracy: 0.9685 - val_loss: 0.0563 - val_accuracy: 0.9854
Epoch 5/100
55000/55000 [==============================] - 12s 226us/sample - loss: 0.1021 - accuracy: 0.9748 - val_loss: 0.0625 - val_accuracy: 0.9848
Epoch 6/100
55000/55000 [==============================] - 13s 227us/sample - loss: 0.0845 - accuracy: 0.9793 - val_loss: 0.0644 - val_accuracy: 0.9882
Epoch 7/100
55000/55000 [==============================] - 13s 230us/sample - loss: 0.0701 - accurac

In [ ]:
# save the obtained output
np.save('output_3.npy', output_3)

In [6]:
# CNN settings 2

runs = 1
output_4 = np.zeros([runs, 3*len(settings_2)])

for ix, s in enumerate(settings_2):
    print("Activation: "+s[0]+". Initialization: "+s[1])
    
    for i in range(runs):
        print("Model iteration: "+str(i))
        model = keras.models.Sequential([
            keras.layers.Conv2D(64, 7, activation=s[0], kernel_initializer=s[1], padding="same", 
                                input_shape=[28, 28, 1]),
            keras.layers.MaxPooling2D(2),
            keras.layers.Conv2D(128, 3, activation=s[0], kernel_initializer=s[1], padding="same"),
            keras.layers.Conv2D(128, 3, activation=s[0], kernel_initializer=s[1], padding="same"),
            keras.layers.MaxPooling2D(2),
            keras.layers.Conv2D(256, 3, activation=s[0], kernel_initializer=s[1], padding="same"),
            keras.layers.Conv2D(256, 3, activation=s[0], kernel_initializer=s[1], padding="same"),
            keras.layers.MaxPooling2D(2),
            keras.layers.Flatten(),
            keras.layers.Dense(128, activation=s[0], kernel_initializer=s[1]),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(64, activation=s[0], kernel_initializer=s[1]),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_uniform")
        ])
        
        model.compile(loss="sparse_categorical_crossentropy",
                      optimizer="sgd",
                      metrics=["accuracy"])
        
        history = model.fit(X_train_, y_train, epochs=2, validation_data=(X_valid_,y_valid),
                            callbacks=[keras.callbacks.EarlyStopping(patience=10)])
        
        output_4[i,ix*3] = model.evaluate(X_test_, y_test)[1]
        output_4[i,3*ix+1] = len(history.history['loss'])
        output_4[i,3*ix+2] = history.history['val_accuracy'][0]

Activation: tanh. Initialization: glorot_uniform
Model iteration: 0
(55000, 28, 28)
Train on 55000 samples, validate on 5000 samples
Epoch 1/2
55000/55000 [==============================] - 9s 158us/sample - loss: 0.4939 - accuracy: 0.8619 - val_loss: 0.0992 - val_accuracy: 0.9708
Epoch 2/2
10000/10000 [==============================] - 1s 68us/sample - loss: 0.4164 - accuracy: 0.8573
Activation: tanh. Initialization: glorot_normal
Model iteration: 0
(55000, 28, 28)
Train on 55000 samples, validate on 5000 samples
Epoch 1/2
55000/55000 [==============================] - 7s 134us/sample - loss: 0.5212 - accuracy: 0.8507 - val_loss: 0.0957 - val_accuracy: 0.9730
Epoch 2/2
10000/10000 [==============================] - 1s 62us/sample - loss: 0.1185 - accuracy: 0.9637
Activation: relu. Initialization: he_uniform
Model iteration: 0
(55000, 28, 28)
Train on 55000 samples, validate on 5000 samples
Epoch 1/2
55000/55000 [==============================] - 7s 134us/sample - loss: 0.7443 - accura

In [ ]:
# save the obtained output
np.save('output_4.npy', output_4)